## Parameters

In [ ]:
prefix='./results/'
n_jobs = 10
n_trees = 10

## Imports

In [ ]:
from metrics import *
%run ___imports.ipynb 

## Loading

In [ ]:
with open('simplified_reccurent_CD_pt1.pkl', "rb") as fh:
            df1 = pickle.load(fh)
with open('simplified_reccurent_CD_pt2.pkl', "rb") as fh:
            df2 = pickle.load(fh)        
with open('simplified_reccurent_CD_pt3.pkl', "rb") as fh:
            df3 = pickle.load(fh)       
with open('simplified_reccurent_CD_pt4.pkl', "rb") as fh:
            df4 = pickle.load(fh)

df = pd.concat([df1,df2,df3,df4])

df.TX_DATETIME = (df.TX_DATETIME - datetime.strptime('2018-04-01', '%Y-%m-%d')).dt.days

train = df[df['TX_DATETIME']>=0]
train = train[train['TX_DATETIME']<7]
test = df[df['TX_DATETIME']>=7]

## Preparation

In [ ]:
fraudulent_card_train = np.unique(train.loc[train.TX_FRAUD == 1]['CUSTOMER_ID'].values)

In [ ]:
performancesMeasures = pd.DataFrame(columns=['model','day_test','pk100','pk300','pk1000'])

## Execution

In [ ]:
ensemble = []

In [ ]:
important_features = train.columns.values.tolist()
to_remove = ['TX_DATETIME','TX_TIME_SECONDS','CUSTOMER_ID', 'TX_FRAUD']
for i in to_remove: important_features.remove(i)

In [ ]:
dataset = train.append(test)
first_day_test = 7
last_day_test = 182

In [ ]:
#Initialization
#We need to fill the buffer with the last 7 windows 

for i in range(first_day_test -4 ,first_day_test):
    training_day_from = i -7
    training_day_to = i
    print("Training day: from "+str(training_day_from)+ " to "+str(training_day_to))
    
    #df because outside of train/test(hence outside of dataset)
    train_set = df[df['TX_DATETIME'] >= training_day_from]
    train_set = train_set[train_set['TX_DATETIME'] < training_day_to]
    
    if i == first_day_test - 4 :
        fraudulent_card_train = np.unique(train_set.loc[train_set['TX_FRAUD'] == 1]['CUSTOMER_ID'].values)
    else:
        fraudulent_card_train = np.unique(np.concatenate([fraudulent_card_train,np.unique(train_set.loc[train_set['TX_FRAUD'] == 1]['CUSTOMER_ID'].values)]))

    Xtrain = train_set[important_features]
    ytrain = train_set['TX_FRAUD']
    
    clf = BalancedRandomForestClassifier(n_jobs=n_jobs,n_estimators=n_trees,random_state=42, verbose=5)
    clf.fit(Xtrain,ytrain)
    
    ensemble.append(clf) #ensemble initialization

In [ ]:
for i in range(first_day_test,last_day_test+1):
    print("TEST DAY "+str(i)+" of "+str(last_day_test))

    training_day_from = i -7
    training_day_to = i
    testing_day = i
    
    print("Training day: from "+str(training_day_from)+ " to "+str(training_day_to))
    print("Testing day: "+str(testing_day))

    train_set = dataset[dataset['TX_DATETIME'] >= training_day_from]
    train_set = train_set[train_set['TX_DATETIME'] < training_day_to]
    
    test_set = dataset[dataset['TX_DATETIME'] == testing_day]
    
    
    #fraudulent_card_train has been initialized during previous phase
    fraudulent_card_train = np.unique(np.concatenate([fraudulent_card_train,np.unique(train_set.loc[train_set['TX_FRAUD'] == 1]['CUSTOMER_ID'].values)]))
    

    Xtrain = train_set[important_features]
    Xtest = test_set[important_features]
    ytrain = train_set['TX_FRAUD']
    ytest = test_set['TX_FRAUD']
 
    
    clf = BalancedRandomForestClassifier(n_jobs=n_jobs,n_estimators=n_trees,random_state=42, verbose=5)
    clf.fit(Xtrain,ytrain)

    ensemble.append(clf)
    #pop a random element
    _ = ensemble.pop(random.randint(0,len(ensemble)-1))
    
    
    all_preds = np.zeros(shape=(Xtest.shape[0],len(ensemble)))
    for j in range(len(ensemble)):
        print ("Predicting for model "+str(j+1)+" of "+str(len(ensemble)))
        all_preds[:,j]=ensemble[j].predict_proba(Xtest)[:,1]

       
    pred_online = np.average(a=all_preds,axis=1)

    
    model_name = prefix+'random_ensemble'
  
    print("Measuring performances for slot: "+str(i))
    
    pk100,pk300,pk1000,cpk100,cpk300,cpk1000,prauc,prauc_card,cprauc,cprauc_card = compute_metrics_remove_cards(ytest,pred_online,test_set['CUSTOMER_ID'],fraudulent_card_train) 
    performancesMeasures.loc[len(performancesMeasures)] = [model_name,
                                                           testing_day,
                                                           pk100,
                                                           pk300,
                                                           pk1000]
      

In [ ]:
perf_name = prefix+'metrics_BRF_random_ensemble.csv'
performancesMeasures.to_csv(perf_name,index=False)
performancesMeasures

In [ ]:
performancesMeasures.pk100.plot()